
### Step 1: Install Required Libraries

In the first cell of your Colab notebook, install the necessary libraries. LangChain and OpenAI are the primary libraries needed.

In [2]:
!pip install langchain_community openai kagglehub

  Using cached langchain_community-0.3.8-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7



- **LangChain**: A framework for building applications with language models.

- **OpenAI**: The library to interact with OpenAI's language models.


### Step 2: Import Libraries and Set Up OpenAI API Key

In the next cell, import the necessary modules and set up your OpenAI API key. You need to replace `"YOUR_API_KEY"` with your actual OpenAI API key.

In [5]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import langchain
import kagglehub
import os
import pandas as pd

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"


- **OpenAI API Key**: This key is required to authenticate your requests to OpenAI's API. You can obtain it from [OpenAI's website](https://auth.openai.com/authorize?issuer=auth0.openai.com&client_id=DRivsnm2Mu42T3KOpqdtwB3NYviHYzwD&audience=https%3A%2F%2Fapi.openai.com%2Fv1&redirect_uri=https%3A%2F%2Fplatform.openai.com%2Fauth%2Fcallback&device_id=70c25419-0a68-4868-aaa0-6ac95051c26a&screen_hint=signup&max_age=0&scope=openid+profile+email+offline_access&response_type=code&response_mode=query&state=WEtzQ3ZIcV9RUW5uNGZYTUc3OU9Lak00fmZHQ3dlMzh%2BNlBZbjh1fi5%2BMA%3D%3D&nonce=Q0Y2Y0hocmpKQXZUWTFNQnNIckpKMFdBSnBXZEM1akF5Y2VkeXE3MWpjOA%3D%3D&code_challenge=cMl19UnF6Bn7h2FKgCKo6Ax_7W8LDvAeIP6PTm_gOC8&code_challenge_method=S256&auth0Client=eyJuYW1lIjoiYXV0aDAtc3BhLWpzIiwidmVyc2lvbiI6IjEuMjEuMCJ9&flow=control).


### Step 3: Set Up The Kaggle Dataset

In the next cell, import the kaggle dataset.

In [9]:
file=kagglehub.dataset_download("niraliivaghani/flipkart-product-customer-reviews-dataset")
print("Path to the file: ",file)

100%|██████████| 3.79M/3.79M [00:00<00:00, 153MB/s]

Extracting files...
Path to the file:  /root/.cache/kagglehub/datasets/niraliivaghani/flipkart-product-customer-reviews-dataset/versions/1


In [12]:
# Find the correct CSV file in the directory
dataset_directory = file # This is the directory returned by kagglehub
csv_files = [f for f in os.listdir(dataset_directory) if f.endswith('.csv')]

if len(csv_files) == 0:
    raise FileNotFoundError("No CSV files found in the dataset directory.")
elif len(csv_files) > 1:
    print("Multiple CSV files found. Using the first one:", csv_files[0])

# Construct the full path to the CSV file
csv_path = os.path.join(dataset_directory, csv_files[0])

# Load the dataset into a pandas DataFrame
df = pd.read_csv(csv_path)
print("Dataset loaded successfully.")


print(df.head(10))


Dataset loaded successfully.
                                        product_name product_price Rate  \
0  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    5   
1  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    5   
2  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    3   
3  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    1   
4  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    3   
5  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    5   
6  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    5   
7  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    3   
8  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    1   
9  Candes 12 L Room/Personal Air Cooler??????(Whi...          3999    4   

               Review                                            Summary  \
0              super!  great cooler excellent air flow and for this p


### Step 4: Define and Run the Chain

In the next cell, define a simple chain using LangChain and run it.

In [19]:
# Simple Chain
prompt = PromptTemplate(
    template="What was the review for the {product_name}?",
)
chain = LLMChain(llm=OpenAI(), prompt=prompt)


In [20]:
# Run the chain
product_name = "Candes 12 L Room/Personal Air Cooler"
chain.invoke(product_name)

{'product_name': 'Candes 12 L Room/Personal Air Cooler',
 'text': '\n\nThe review for the Candes 12 L Room/Personal Air Cooler was mostly positive. Users praised its compact and portable design, making it easy to move from room to room. They also appreciated its strong cooling performance, even in hot and humid weather. The air cooler was also noted for its low noise level and energy efficiency. However, some users mentioned that the water tank could have been larger and that the cooler was not very effective in larger rooms. Overall, the majority of reviewers were satisfied with their purchase and would recommend it to others.'}


- **Prompt**: The input text that you provide to the language model. In this case, it's a request to generate a marketing email.

- **LLMChain**: A LangChain object that manages the interaction with the language model. It takes a language model (`llm`) and a `prompt`.

- **Run the Chain**: The `run` method executes the chain and returns the response from the language model.


### Additional Resources

- **LangChain Documentation**: [LangChain Docs](https://langchain.readthedocs.io/en/latest/)

- **OpenAI API Documentation**: [OpenAI API Docs](https://platform.openai.com/docs/api-reference/introduction)